In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_paths = [
    ('img/l1.png', 'img/r1.png'),
    ('img/l2.png', 'img/r2.png'),
    ('img/l3.png', 'img/r3.png'),
]

In [ ]:
WINDOW_SIZES = [1, 5, 9]
DISPARITY_RANGE = 64

In [5]:
def load_images(left_img_path, right_img_path):
    left = cv2.imread(left_img_path, cv2.IMREAD_GRAYSCALE)
    right = cv2.imread(right_img_path, cv2.IMREAD_GRAYSCALE)
    return left, right

In [ ]:
def compute_sad_ssd(left, right, window_size, method='SAD'):
    h, w = left.shape
    disparity_map = np.zeros((h, w), dtype=np.uint8)
    half_w = window_size // 2
    for y in range(half_w, h - half_w):
        for x in range(half_w, w - half_w):
            min_cost = float('inf')
            best_disparity = 0
            for d in range(DISPARITY_RANGE):
                if x - d - half_w < 0:
                    continue
                left_patch = left[y - half_w:y + half_w + 1, x - half_w:x + half_w + 1]
                right_patch = right[y - half_w:y + half_w + 1, x - d - half_w:x - d + half_w + 1]
                if method == 'SAD':
                    cost = np.sum(np.abs(left_patch - right_patch))
                elif method == 'SSD':
                    cost = np.sum((left_patch - right_patch) ** 2)
                if cost < min_cost:
                    min_cost = cost
                    best_disparity = d
            disparity_map[y, x] = best_disparity * (255 // DISPARITY_RANGE)
    return disparity_map

In [ ]:
img_id = 0
def main():
    left, right = load_images(image_paths[img_id][0],image_paths[img_id][1])
    for w in WINDOW_SIZES:
        print(f'Computing disparity map for window size {w}...')
        print('SAD method...')
        sad_map = compute_sad_ssd(left, right, w, method='SAD')
        print('SSD method...')
        ssd_map = compute_sad_ssd(left, right, w, method='SSD')
        cv2.imwrite(f'disparity/disparity_imag_{img_id}_sad_w{w}.png', sad_map)
        cv2.imwrite(f'disparity/disparity_imag_{img_id}_ssd_w{w}.png', ssd_map)

if __name__ == '__main__':
    main()